In [1]:
from model.types import MarketDemandSupply
from model.run import load_config, run, post_processing
import matplotlib.pyplot as plt
from model.mechanism.amm_k import amm_k_mechanism
from model.mechanism.treasury import liq_backing_mechanism,treasury_stables_mechanism,liq_ohm_mechanism
from model.mechanism.protocol import floating_supply_mechanism,mcap_mechanism,ratio_mechanism
from model.policy.rbs_price import lower_target_policy,upper_target_policy
from model.policy.ohmbond import generate_ohmbond


params = {"demand_factor": [0.007, .003, .001],
         "supply_factor": [-0.007, -.003, -.001],
         "initial_reserves_volatile":[25000000], # setting it as constant since in this simulation we don't use it at all.
         "max_liq_ratio":[0.14375],  # LiquidityUSD : reservesUSD ratio --> 1:1 = 0.5
         "target_ma":[30], # number of days
         "lower_wall":[0.15],
         "upper_wall":[0.15],
         "lower_cushion":[.075],
         "upper_cushion":[.075],
          "reinstate_window": [30],
          "max_outflow_rate": [0.05],
          "reward_rate_policy": ["Flat"],
          "ask_factor": [0.095],  # % of floating supply that the treasury can deploy when price is trading above the upper target
          "bid_factor": [0.095],  # % of the reserves that the treasury can deploy when price is trading below the lower target
         "cushion_factor": [0.3075],  # The percentage of a bid or ask to offer as a cushion
        "min_counter_reinstate": [6],  # Number of days within the reinstate window that conditions are true to reinstate a bid or ask
         "with_reinstate_window": ['Yes'],
         "bond_create_schedule":[generate_ohmbond(amounts=[[1e6]*3],exp_durs=[[30,60,90]])]
         }


initialstate = { # variable values borrowed from liquidity-olympus/simulation.ipynb
            "liq_stables": 21000000,
            "reserves_stables":170000000,
            "reserves_volatile":25000000,
            "price":9.5,
            "reward_rate": 0.000198,

            "market_demand_supply": MarketDemandSupply(total_supply = .001,
                                                   total_demand = -0.008 ), # factors, % of OHM supply expected to drive market demand
                                                   
            # treasury policy
            'target_liq_ratio_reached':False,
            'reserves_in':0,
            'supply':25000000,

            # RBS related
            'ma_target':9.5,
            # market transaction variables
            'ask_change_ohm':0,
            'bid_change_ohm':0,
            "net_flow": None,
    "bid_capacity_target": None,
    "ask_capacity_target": None,
    "bid_capacity_target_cushion": None,
    "ask_capacity_target_cushion": None,
    "natural_price": None,
    "bid_capacity_cushion": None,
    "ask_capacity_cushion": None,
    "bid_change_cushion_usd": None,
    "bid_change_cushion_ohm": None,
    "ask_change_cushion_usd": None,
    "ask_change_cushion_ohm": None,
    "bid_change_usd": None,
    "ask_change_usd": None,
    "reserves_out": None,
    "ohm_traded": None,
    "cum_ohm_purchased": 0,
    "cum_ohm_burnt": 0,
    "cum_ohm_minted": 0
    
            }
# initialize parameters that are decided by other parameters
initialstate['amm_k'] = amm_k_mechanism(initialstate)
initialstate['price_history']=[initialstate['price']]

initialstate['lb_target'] = initialstate['ma_target']
initialstate['price_target']=initialstate['ma_target']

initialstate['liq_ohm'] = liq_ohm_mechanism(initialstate['liq_stables'],initialstate['price'])
initialstate['floating_supply'] = floating_supply_mechanism(initialstate['supply'],initialstate['liq_ohm'])


initialstate['treasury_stables'] = treasury_stables_mechanism(initialstate['liq_stables'],initialstate['reserves_stables'])
initialstate['liq_backing'] = liq_backing_mechanism(initialstate['treasury_stables'],params['initial_reserves_volatile'][0])
initialstate['lower_target_wall'] = lower_target_policy(initialstate['price_target'], params['lower_wall'][0])
initialstate['upper_target_wall'] = upper_target_policy(initialstate['price_target'], params['upper_wall'][0])
initialstate["lower_target_cushion"] = lower_target_policy(initialstate['price_target'], params['lower_cushion'][0])
initialstate["upper_target_cushion"] = upper_target_policy(initialstate['price_target'], params['upper_cushion'][0])

initialstate['bid_counter'],initialstate['ask_counter'] = [0] * params["reinstate_window"][0],[0]* params["reinstate_window"][0]


initialstate["bid_capacity_target"] = params["bid_factor"][0] * initialstate["reserves_stables"]
initialstate["ask_capacity_target"] = params["ask_factor"][0] * initialstate["reserves_stables"] / initialstate["upper_target_wall"] * (1 + params["lower_wall"][0] + params["upper_wall"][0])
initialstate["bid_capacity_target_cushion"] = initialstate["bid_capacity_target"] * params["cushion_factor"][0]
initialstate["ask_capacity_target_cushion"] = initialstate["ask_capacity_target"] * params["cushion_factor"][0]
initialstate["bid_capacity"] = initialstate["bid_capacity_target"]
initialstate["ask_capacity"] = initialstate["ask_capacity_target"]
initialstate["bid_capacity_cushion"] = initialstate["bid_capacity_target_cushion"]
initialstate["ask_capacity_cushion"] = initialstate["ask_capacity_target_cushion"]
initialstate["ask_change_ohm"] = 0
initialstate["bid_change_ohm"] = 0

# ohm bond related variables
initialstate['bond_created']=[]
initialstate['bond_created_today']=[]
initialstate['ohm_bonded']=0
initialstate['liq_ohm_into_bond']=0
initialstate['ohm_released']=0

# protocol vars
initialstate['mcap'] = mcap_mechanism(initialstate['supply'],initialstate['price'])
initialstate['floating_mcap']=mcap_mechanism(initialstate['floating_supply'],initialstate['price'])
initialstate['liq_ratio']=ratio_mechanism(initialstate['liq_stables'],initialstate['treasury_stables'])
initialstate['reserves_ratio']=ratio_mechanism( initialstate['reserves_stables'], initialstate['liq_stables']),
initialstate['fmcap_treasury_ratio']=ratio_mechanism( initialstate['floating_mcap'],initialstate['treasury_stables'])
initialstate['liq_fmcap_ratio']=ratio_mechanism(initialstate['liq_stables'] ,initialstate['floating_mcap'])
initialstate['total_demand']=initialstate['market_demand_supply'].total_demand
initialstate['total_supply']=initialstate['market_demand_supply'].total_supply
initialstate['total_net']=initialstate['total_demand']+initialstate['total_supply']

# Param tracking
initialstate['demand_factor'] = None
initialstate['supply_factor'] = None

nrun = 2
nstep = 100
exp = load_config(nrun, params, initialstate, nstep)
raw = run(exp)
df = post_processing(raw)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 100, 18, 6, 63)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (100, 18, 6, 63)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 24.41s


In [2]:
df["unique_id"] = df["simulation"].astype(str) + "-" + df["subset"].astype(str)+  "-" +df["run"].astype(str)

In [3]:
from typing import List
from pandas import DataFrame

def plot_all_sims(var_list: List[str], df: DataFrame):
    for col in var_list:
        df.pivot("timestep","unique_id", col).plot(kind='line')
        plt.title(col)
        plt.legend([])
        plt.show()
        
def plot_grouped_variables_average(var_list: List[str], grouping_variables: List[str], df: DataFrame):
    data = df.groupby(grouping_variables)[var_list + ["timestep"]].mean()
    print(data)
    for col in var_list:
        data.pivot("timestep","unique_id", col).plot(kind='line')
        plt.title(col)
        plt.legend([])
        plt.show()

In [4]:
df

,simulation,subset,run,timestep,liq_stables,reserves_stables,reserves_volatile,price,reward_rate,market_demand_supply,...,liq_fmcap_ratio,total_demand,total_supply,total_net,demand_factor,supply_factor,substep,control_ask,control_bid,unique_id
0,0,0,1,0,2.100000e+07,1.700000e+08,25000000,9.500000,0.000198,"MarketDemandSupply(total_supply=0.001, total_d...",...,0.096998,-0.008000,0.001000,-0.007000,None,None,0,0,0,0-0-1
1,0,0,1,1,2.054378e+07,1.700000e+08,25000000,9.091709,0.000198,MarketDemandSupply(total_supply=-0.01008115111...,...,0.099344,0.008189,-0.010081,-0.001893,None,None,23,0,0,0-0-1
2,0,0,1,2,2.174620e+07,1.700730e+08,25000000,10.187125,0.000198,MarketDemandSupply(total_supply=-0.01758475730...,...,0.093318,0.005974,-0.017585,-0.011611,None,None,23,1,0,0-0-1
3,0,0,1,3,2.174843e+07,1.706396e+08,25000000,10.189215,0.000198,MarketDemandSupply(total_supply=-0.01893337982...,...,0.093258,0.016711,-0.018933,-0.002222,None,None,23,1,1,0-0-1
4,0,0,1,4,1.935909e+07,1.701977e+08,25000000,8.073372,0.000198,MarketDemandSupply(total_supply=-0.02048812571...,...,0.105705,0.004160,-0.020488,-0.016329,None,None,23,1,2,0-0-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,0,2,2,96,2.725487e+07,1.622562e+08,25000000,16.001964,0.000198,MarketDemandSupply(total_supply=-0.00070799479...,...,0.053786,0.001743,-0.000708,0.001035,None,None,23,0,30,0-2-2
602,0,2,2,97,2.750597e+07,1.622562e+08,25000000,16.298175,0.000198,MarketDemandSupply(total_supply=-0.00220085872...,...,0.053258,0.002302,-0.002201,0.000101,None,None,23,0,30,0-2-2
603,0,2,2,98,2.710081e+07,1.625200e+08,25000000,15.821574,0.000198,MarketDemandSupply(total_supply=-0.00114153525...,...,0.054086,0.001356,-0.001142,0.000215,None,None,23,0,30,0-2-2
604,0,2,2,99,2.723422e+07,1.625200e+08,25000000,15.977725,0.000198,MarketDemandSupply(total_supply=-0.00230839223...,...,0.053824,0.001372,-0.002308,-0.000937,None,None,23,0,30,0-2-2


In [5]:
# If len(params) > 1: include in output

In [6]:
plot_grouped_variables_average(["reserves_in","reserves_out"],
                               ["demand_factor", "supply_factor"],
                               df)

Empty DataFrame
Columns: [reserves_in, reserves_out, timestep]
Index: []


KeyError: 'unique_id'

In [ ]:
plot_all_sims(["reserves_in","reserves_out"], df)